# Library

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%cd /content/gdrive/My Drive/dark/darkflow-master
!ls

/content/gdrive/My Drive/dark/darkflow-master
bin	  darkflow.egg-info  fooball_v2.mp4   labels.txt      README.md
build	  data		     fooball_v3.mp4   LICENSE	      sample_img
cfg	  demo.gif	     fooball_v4.mp4   nearside.jpg    setup.py
ckpt	  flow		     fooball_v5.mp4   new_model_data  test
darkflow  fooball_v1.avi     football_v6.mp4  preview.png     YOLO.ipynb


In [0]:
# %cd /content/gdrive/My Drive/Colab Notebooks/YOLO/Yolo(Darkflow)/darkflow-master
# !ls

In [0]:
import numpy as np
import cv2
from darkflow.net.build import TFNet
from matplotlib import pyplot as plt
import time
from IPython.display import clear_output
import os
from pathlib import Path
from scipy.spatial.distance import euclidean

%matplotlib inline

# Custom Function and Contants

# Capture frames from the video for training (filter some of the repeated frames) Only select wide shot

In [0]:
def frame_selection(video_path, step = 10):
  capture = cv2.VideoCapture(video_path)
  counter = 0
  pic_number = 0 
  while capture.isOpened():
    starttime = time.time()
    status, frame = capture.read()
  #   print(counter)
    if counter % step == 0:
      print(counter)
      print('\nrunning')

      try:
        results = tfnet.return_predict(frame)
      except:
        print('No Frame')
        
      
      if status:
        ranks = np.array([])
        for result in results:
          ranks = np.insert(ranks, len(ranks), result['confidence'])
        ranks = np.argsort(ranks)[::-1]
        
        for r in ranks:
          if results[r]['label'] == 'person':
            print(results[r]['label'])
            confidence_level = results[r]['confidence']
            
            area = int((results[r]['bottomright']['x'] - results[r]['topleft']['x']) * \
              (results[r]['bottomright']['y'] - results[r]['topleft']['y']))
            print(area)
            if area > 2000 and confidence_level > 0.6:
              break
              print('Close Shot')
              
            elif area <= 2000:
              print('save image')
              cv2.imwrite(f'./data/images/{pic_number}.png', frame)
  #                         [cv2.IMWRITE_PNG_COMPRESSION, 0])
              pic_number +=1
              break
              
          else:
            continue
          
    

        print('fps: %0.2f'%(1 / (time.time() - starttime)))
        time.sleep(0.1)
        clear_output(wait = True)
      else:
        capture.release()
        cv2.destroyAllWindows
        print(f'Total Frame saved: {pic_number}')
        break
    counter += 1
  #   if counter == total_frame:
  #     break
  print(f'Total Frame: {counter}')

# Color

In [0]:
def point_coordinate(pt1,pt2,m,n):
  # pt1 is self
  new_x = (m*pt2[0]+n*pt1[0])/(m+n) 
  new_y = (m*pt2[1]+n*pt1[1])/(m+n)
  return(int(new_x),int(new_y))

In [0]:
color_dict={
    'black': {'high': np.array([141, 207,  97]),
  'low': np.array([132,  87,  46])},
    
 'green': {'high': np.array([ 58, 136, 171]),
          'low': np.array([ 50,  75, 136])},
    
 'red': {'high': np.array([176, 255, 255]),
  'low': np.array([169, 138, 197])},
    
 'white&blue': {'high': np.array([117,  42, 255]),
              'low': np.array([  0,   0, 217])},
 'yellow': {'high': np.array([ 39, 227, 255]),
            'low': np.array([ 30,  98, 166])}
           }

rectangle_color={
            'yellow' : (36,242,242),
    
           'black' : (0,0,0),
    
           'red' : (10,56,240),
    
           'white&blue':(237,237,206)
    
}

In [0]:
def color_detect(img,info,color1,color2,color3,color4,color_dict):
      if color(img,info,standard=2000,Color=color1,low=color_dict[color1]['low'],
               high=color_dict[color1]['high'])==color1:

        Color=color1
      elif color(img,info,standard=5000,Color=color2,low=color_dict[color2]['low'],
               high=color_dict[color2]['high'])==color2:

        Color=color2
      elif color(img,info,standard=5000,Color=color3,low=color_dict[color3]['low'],
               high=color_dict[color3]['high'])==color3:

        Color=color3
      # grass
      elif color(img,info,standard=5000,Color='green',low=color_dict['green']['low'],
               high=color_dict['green']['high'])=='green':

        Color='green'
      else:
        Color=color4
      return Color

In [0]:
# function
# final!
def color(img,info,standard=10000,Color='yellow',low=np.array([30,103,195]),high=np.array([33,227,255])):
#   print(Color)
  person=cut_person(img,info)
  hsv = cv2.cvtColor(person, cv2.COLOR_BGR2HSV)
  mask =cv2.inRange(hsv,low,high)
#   cv2_imshow(mask)
  if Color=='green':
    standard=(person.shape[0]*person.shape[0])*(255)*0.7
  if sum(mask.flatten())>standard:
#     print(f'{color} ',sum(mask.flatten()))
    return Color
  else:
#     print(f'not {color}',sum(mask.flatten()))
    return f'not {Color}'

In [0]:
def cut_person(img,info):
  col2=info['bottomright']['x']
  row2=info['bottomright']['y']
  col1=info['topleft']['x']
  row1=info['topleft']['y']
  person= img[row1:row2,col1:col2]
  return person

In [0]:
# latest function
# What is the value?
def low_and_high(target,low='?',high='?'):
  hsv_range=cv2.cvtColor(target, cv2.COLOR_BGR2HSV).reshape(int(len(target.flatten())/3),1,3)
  h_list=[]
  s_list=[]
  v_list=[]
  print(f'old(high,low): {(high,low)}')
  for hsv in hsv_range:
    [[h,s,v]] =hsv
    h_list.append(h)
    s_list.append(s)
    v_list.append(v)


  try: 
    if low[0]>min(h_list):  
      low[0]=min(h_list)
    if low[1]>min(s_list):
      low[1]=min(s_list)
    if low[2]>min(v_list):
      low[2]=min(v_list)
    if high[0]<max(h_list):
      high[0]=max(h_list)
    if high[1]<max(s_list):
      high[1]=max(s_list)
    if high[2]<max(v_list):
      high[2]=max(v_list)
    print(f'New(high,low): {(high,low)}')
  except:
    low=np.array([min(h_list),min(s_list),min(v_list)])
    high=np.array([max(h_list),max(s_list),max(v_list)])
    print(f'Make New(high,low): {(high,low)}')
  return(high,low)

# Output Video(s)

In [0]:
options = {
    'model' : './cfg/yolov2-voc-footballv2.cfg',
    'load' : 7125,
    'threshold' : 0.6,
    'gpu': 1.0
}

tfnet = TFNet(options)


def outputvideo(target_video_path, save_video_name, color1='yellow',color2='black',color3='red',color4='white&blue',color_dict=color_dict):
  # video_output = []
  capture = cv2.VideoCapture(target_video_path)
  # print(capture)

  fourcc = cv2.VideoWriter_fourcc(*'XVID')
  fps = capture.get(cv2.CAP_PROP_FPS)
  print(fps)
  size = (int(capture.get(cv2.CAP_PROP_FRAME_WIDTH)), int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT)))
  print(size)
  output_video = cv2.VideoWriter(save_video_name, fourcc, fps, size) 

  # color_label = ['Player', 'Football', 'Goalpost', 'Referee', 'Holding the ball']
  # color_code = [tuple(255 * np.random.rand(3)) for i in range(len(color_label))]
  # color_dict = dict(zip(color_label, color_code))


  while (capture.isOpened()):
      starttime = time.time()
      ret, frame = capture.read()

      try:
        frame_g = frame.copy()
        frame_g = cv2.cvtColor(frame_g, cv2.COLOR_BGR2GRAY)
        frame_g = np.expand_dims(frame_g, 2) #add 3rd dimension
        frame_g = np.insert(frame_g, 1, [frame[:,:,0]] * 2, axis = 2) #change 1 channel to 3 channel
        results = tfnet.return_predict(frame_g) #return result
      except:
        print('No array!!!!')
      print(ret)

      if ret:
        center_list = []
        distance_list = []
        distance_list_team1 = []
        distance_list_team2 = []
        center_list_team1 = []
        center_list_team2 = []
        for result in results:
          try:
            Color=color_detect(frame,result,color1='yellow',color2='black',color3='red',color4='white&blue',color_dict=color_dict)
            tlx, tly = (result['topleft']['x'], result['topleft']['y'])
            brx, bry =  (result['bottomright']['x'], result['bottomright']['y'])
            center_x = int((tlx + (brx - tlx)/2 ))
            center_y = int((tly + (bry - tly) / 2) )
            if Color == 'yellow':
              center_list_team1.append((center_x, center_y))
            if Color == 'white&blue':
              center_list_team2.append((center_x, center_y))

          except:
            continue

        for center in range(len(center_list_team1)):
          center_list_ = center_list_team1.copy()
          pop_value = center_list_.pop(center)
        #   print(center_list_)
          temp = dict(zip(map(lambda x: euclidean(pop_value,x),center_list_), center_list_))
          temp_key = sorted([key for key in temp.keys()])[:2]

          try: 
            distance_list_team1.append([temp[temp_key[0]], temp[temp_key[1]]])
          except:
            distance_list_team1.append([0, 0])


        for center in range(len(center_list_team2)):
          center_list_ = center_list_team2.copy()
          pop_value = center_list_.pop(center)
        #   print(center_list_)
          temp = dict(zip(map(lambda x: euclidean(pop_value,x),center_list_), center_list_))
          temp_key = sorted([key for key in temp.keys()])[:2]
          try:
            distance_list_team2.append([temp[temp_key[0]], temp[temp_key[1]]])
          except:
            distance_list_team2.append([0, 0])


        for result in range(len(results)):
          try:
            Color=color_detect(frame,results[result],color1='yellow',color2='black',color3='red',color4='white&blue',color_dict=color_dict)
            tl = (results[result]['topleft']['x'], results[result]['topleft']['y'])
            br = (results[result]['bottomright']['x'], results[result]['bottomright']['y'])
            label = results[result]['label']
            color_label = rectangle_color[Color]


            if label == 'Football':
              color_label = (162,130,122) #purple, BGR

            confidence_label = results[result]['confidence']
            text = '%s: %.2f%%'%(label, confidence_label * 100)
            frame = cv2.rectangle(frame, tl, br, color_label, 2)
            frame = cv2.putText(frame, text, tl, cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0), 2)

            if Color == 'yellow':
              center_list = center_list_team1
              distance_list = distance_list_team1
              color_arrow = (178,102,255)#pink
  #             print(point_coordinate(center_list[result], distance_list[result][1], 4, 6))
            if Color == 'white&blue':
              center_list = center_list_team2
              distance_list = distance_list_team2
              color_arrow = (255,178,102)#light blue
  #             print(point_coordinate(center_list[result], distance_list[result][1], 4, 6))
            if Color == 'yellow' or Color == 'white&blue':
              ratio_m = 0
              ratio_n = 0
              for i in range(2):
                distance_with_teammate = euclidean(center_list[result], distance_list[result][i])
                if  distance_with_teammate >= 50 and distance_with_teammate <= 150:
                  ratio_m = 2
                  ratio_n = 8
                elif distance_with_teammate > 150:
                  ratio_m = 1
                  ratio_n = 9
                else:
                  ratio_m = 3
                  ratio_n = 7
                frame = cv2.arrowedLine(frame, center_list[result], 
                                        point_coordinate(center_list[result], 
                                        distance_list[result][0], ratio_m, ratio_n),color_arrow,2) 
                frame = cv2.arrowedLine(frame, center_list[result], 
                                        point_coordinate(center_list[result], 
                                        distance_list[result][1], ratio_m, ratio_n),color_arrow,2) 

          except:
            continue


        output_video.write(frame)
  #         plt.imshow(frame)
  #         plt.show()
  #       video_output.append(frame)
  #         clear_output(wait = True)
        print('FPS %.1f'%(1 / (time.time() - starttime)))
  #         if cv2.waitKey(1) & 0xFF == ord('q'):
  #             break
      else:
          output_video.release()
          capture.release()
          cv2.destroyAllWindows()
          break
    

Parsing ./cfg/yolov2-voc-footballv2.cfg
Loading None ...
Finished in 0.000102996826171875s

Building net ...

Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------



       |        | input                            | (?, 416, 416, 3)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 416, 416, 32)

 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 208, 208, 32)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 208, 208, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 104, 104, 64)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 128)
 Init  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 104, 104, 64)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 52, 52, 128)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 52, 52, 256)
 Init  |  Yep!  | conv 1x1p0_1  +bnorm  leak

In [0]:
outputvideo('./data/(cut)Full Match Brazil vs Argentina 2018 Fifa World Cup Qualifiers 11 10 2016 YouTube.mp4', 'football_v7.mp4')

30.0
(1280, 720)
True
FPS 0.8
True
FPS 1.6
True
FPS 1.8
True
FPS 1.8
True
FPS 1.7
True
FPS 1.7
True
FPS 1.8
True
FPS 1.8
True
FPS 1.7
True
FPS 1.7
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.7
True
FPS 1.8
True
FPS 1.8
True
FPS 1.6
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.7
True
FPS 1.8
True
FPS 1.8
True
FPS 1.6
True
FPS 1.7
True
FPS 1.6
True
FPS 1.7
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.9
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.7
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.8
True
FPS 1.7
True
FPS 1.8
True
FPS 1.8
True
FPS